In [4]:
import torch
import numpy as np
from torch import nn

## ScaledDotProductAttentionの実装
---
### 概要
---
qは***[batch,T,d_model]***の次元を持っている。
qはhで割られて分割されるので分割された後[h,batch,T,d_k]の次元になる。この時h×d_k=d_modelとなる。
このd_kを使ってスケーリングをする。


### attention_maskの実装
---
attention_weightに対して同じ形状のmaskは未来の情報を参照しないように上三角行列のようなマスクをかける。
tensor.masked_fill_()メソッドは元テンソルとmaskの形状が同じ時maskのTrueの部分に対して第二引数に与えられた値で元テンソルを埋める。
今回はその値がfloatで指定できる一番小さい値であるためsoftmaxをかけた時0に行くようになっている。


### logitの作成
---
これは見慣れたいつもの式でDotProductattentionを作成する。
```
torch.matmul(q,torch.transpose(k,1,2))/d_k**(0.5)
```


### softmaxを通す
---
これも見慣れた式でやる。
```
nn.functional.softmax(logit,dim=2)
```


### vと掛け算する。
---
これもいつも通り
```
torch.matmul(attention_weight,v)
```

In [5]:
class ScaledDotProductAttention(nn.Module):
    def __init__(
        self,
        d_k:int
    ) -> None:
        super().__init__()
        self.d_k = d_k
    def forward(
        self,
        q:torch.Tensor,
        k:torch.Tensor,
        v:torch.Tensor,
        mask:torch.Tensor = None,
    ) -> torch.Tensor:
        
        ###次元の調節
        scaler = self.d_k**(0.5)
        ###attention_weightの作成。
        logit = torch.matmul(q,torch.transpose(k,1,2))/scaler
        ###maskの作成
        ###maskがNoneじゃなかったらshapeの確認
        if mask is not None:
            if mask.dim() != logit.dim():
                print("mask must have same dim with attention_weight")
            else:
                with torch.no_grad():
                    attention_weight = logit.masked_fill_(
                      mask,
                      -torch.finfo(torch.float).max
                    )
        attention_weight = nn.functional.softmax(logit,dim=2)
        return torch.matmul(attention_weight,v)

## MultiHeadAttentionの実装
---
### 概要
---
MHAの実装。

### 実装手順
---
1.  qをnum_head数重ねてQにする。qは(num_head,batch_size,seq_len,d_model)
2.  QとWをかけて回転行列の積にする。この時h次元に干渉しないようにeinsumを使う。
3.  reshapeを使ってd_modelを分割する。reshapeはd_modelをh個に分割してd_k次元にする。例えば  
    ```
    [[[0,1,2,3,4,5]
      [6,7,8,9,10,11]]]
    ```
    のテンソルがあったとして
    ```
    [[[0,1,2],
      [3,4,5]],
     [[6,7,8],
      [9,10,11]]]
    ```
    のように分割される。
    ((1,2,6)→(2,2,3)の分割)
4.  chunkとcatを使って出力を調整する。
5.  線形層を通して出力を得る。


In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(
        self,
        num_head:int,
        d_model:int
    ) -> None:
        super().__init__()
        self.num_head = num_head
        self.d_model = d_model
        self.d_k = d_model//num_head
        
        self.W_q = nn.Parameter(
            torch.empty(num_head, d_model, self.d_k)
        )
        self.W_k = nn.Parameter(
            torch.empty(num_head, d_model, self.d_k)
        )
        self.W_v = nn.Parameter(
            torch.empty(num_head, d_model, self.d_k)
        )
        nn.init.xavier_uniform_(self.W_q)
        nn.init.xavier_uniform_(self.W_k)
        nn.init.xavier_uniform_(self.W_v)
        
        self.spa = ScaledDotProductAttention(self.d_k)
        self.linear = nn.Linear(self.num_head*self.d_k,self.d_model)
    def forward(
        self,
        q:torch.Tensor,
        k:torch.Tensor,
        v:torch.Tensor,
        mask:torch.Tensor = None
    ) -> torch.Tensor:
        batch_size,seq_len,_ = q.shape
        ###h個にq,k,vを複製。
        Q_h = q.repeat(self.num_head,1,1,1)
        K_h = k.repeat(self.num_head,1,1,1)
        V_h = v.repeat(self.num_head,1,1,1)
        ###パラメータを通して回転行列をかける。
        WQ_h = torch.einsum("hijk,hkl->hijl",(Q_h,self.W_q))
        WK_h = torch.einsum("hijk,hkl->hijl",(K_h,self.W_k))
        WV_h = torch.einsum("hijk,hkl->hijl",(V_h,self.W_v))
        ###d_model次元をd_kに減らしてその分一番上の次元の個数増やしとく。
        WQ_h = torch.reshape(WQ_h,(self.num_head*batch_size,seq_len,self.d_k))
        WK_h = torch.reshape(WK_h,(self.num_head*batch_size,seq_len,self.d_k))
        WV_h = torch.reshape(WV_h,(self.num_head*batch_size,seq_len,self.d_k))
        ###maskもhead数分増やしとく
        if mask is not None:
            mask = mask.repeat(self.num_head,1,1)
        ###attentionの計算をbatch×head数分行う。
        attention_output = self.spa(WQ_h,WK_h,WV_h,mask)
        ###全部のattention計算終わったら最初の次元をnum_head個に分割しておく。
        ###[batch_size,seq_len,self.d_k]の出力がhead数分出来上がる。
        attention_output = torch.chunk(attention_output,self.num_head,dim=0)
        ###[batch_size,seq_len,self.d_k×num_head]次元のテンソルにする
        attention_output = torch.cat(attention_output,dim=2)
        ###線形層を通す。
        output = self.linear(attention_output)

        return output